# Optimization With Numba
My approach so far has focused on working out the details of numpy, but there seems to be a fundamental limit to what I can achieve that way. On the other hand, there seems to be a small side issue that I noted in my last update: 

> I found a bug in the output of my `compute_weights` function: `p_theta_minus_q_theta` has the wrong shape, 20 x 100k instead of 1 x 100k. Obviously my hope is that this reflects an opportunity to further optimize compute weights, but it's probably a shape issue with a sum I did at some point in the calculation.

My next increment for PCITpy should be to resolve this issue and explore the use of numba (or, failing that, Cython) to address these sorts of bottlenecks in my code. I'll also ditch use of `float32` for optimization, as it certainly does change the behavior of my program.

## Setup
Loading relevant packages and data.

In [6]:
import numpy as np
from scipy import special
from numba import jit
import math

tau = .05
bounds = np.array([[-1,  1],
       [ 0,  1],
       [ 0,  1],
       [-1,  1],
       [-1,  1],
       [-1,  1]])
which_param = 0

nth_grp_lvl_param = np.load('../../nth_grp_lvl_param.npy')
nth_prev_iter_curve_param = np.load('../../nth_prev_iter_curve_param.npy')

## Starting Point
The current version of the function prior to further optimization.

In [2]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.divide(x - mu, tau/np.sqrt(.5)) ** 2)
    return log_likelihood

compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

26.8 ms ± 640 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Just Adding the Decorator

In [3]:
%%timeit 
@jit(nopython=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.divide(x - mu, tau/np.sqrt(.5)) ** 2)
    return log_likelihood

compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mUnknown attribute 'erfc' of type Module(<module 'scipy.special' from 'c:\\users\\gunnj\\miniconda3\\lib\\site-packages\\scipy\\special\\__init__.py'>)
[1m
File "<magic-timeit>", line 11:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m
[0m[1mDuring: typing of get attribute at <magic-timeit> (11)[0m
[1m
File "<magic-timeit>", line 11:[0m
[1m<source missing, REPL/exec in use?>[0m


`numba` complains about `special.erfc`. Kind of odd! But resources like [this one](https://github.com/numba/numba/issues/3086) suggest that `numba` just doesn't support that function. Okay. It does apparently support math.erfc. Should I try it?

### Comparing math and special erfc

In [4]:
%%timeit
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    special.erfc(-np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))
    
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

6.36 µs ± 165 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
%%timeit
import math
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    math.erfc(-np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))
    
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

TypeError: only size-1 arrays can be converted to Python scalars

`math.erfc` is not vectorized. I need a vectorized version of `erfc`. Apparently I'm able to make one. Okay...

In [7]:
from numba import vectorize, float64, njit

@vectorize([float64(float64)])
def erfc(x):
    return math.erfc(x)

In [10]:
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    erfc(-np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))
    
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

In [11]:
%%timeit
    
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

5.8 µs ± 110 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


My vectorized `erfc` function is somewhat faster than the original `special.erfc` function.

## Decorator With Vectorized math.erfc

In [68]:
import time

@njit(float64[:,:](float64[:,:], float64[:]), nogil=True,parallel=True, fastmath=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.008000850677490234
Elapsed (after compilation) = 0.011001825332641602


In [69]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

7.47 ms ± 81.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Without parallelization, it takes a lot longer - almost ten times! With parallelization, it's 32 ms, compared to my original 26ms. And `fastmath` does nothing. Neither does `nogil`. Neither does eager compilation (telling Numba the function signature I'm expecting). How odd! Stuff that causes a switch to object mode actually speeds up my code, sadly. 

When I switch back to numpy functions for my squaring, square roots, etc, performance improves substantially. It seems I should almost default to numpy when using jit. Runtime is now 7ms!

# Review
Let's reconsider all those settings I explored for this.

## Vanilla njit

In [3]:
import time 

@njit
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.6515004634857178
Elapsed (after compilation) = 0.22950029373168945


In [4]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

232 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Just One Option

### njit with eager compilation alone

In [5]:
@njit(float64[:,:](float64[:,:], float64[:]))
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.23350048065185547
Elapsed (after compilation) = 0.23749947547912598


In [6]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

261 ms ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### njit with parallel alone

In [7]:
@njit(parallel=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.8864912986755371
Elapsed (after compilation) = 0.008002996444702148


In [8]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

10 ms ± 421 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### njit with nogil alone

In [9]:
@njit(nogil=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.5515000820159912
Elapsed (after compilation) = 0.23650074005126953


In [10]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

236 ms ± 4.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### njit with fastmath alone

In [11]:
@njit(fastmath=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.5214905738830566
Elapsed (after compilation) = 0.22350025177001953


In [12]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

225 ms ± 2.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Two Options

### njit with eager compilation and nogil

In [13]:
@njit(float64[:,:](float64[:,:], float64[:]), nogil=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.23055243492126465
Elapsed (after compilation) = 0.231001615524292


In [14]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

249 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### njit with eager compilation and parallel

In [15]:
@njit(float64[:,:](float64[:,:], float64[:]), parallel=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.009999752044677734
Elapsed (after compilation) = 0.008999109268188477


In [16]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

9.03 ms ± 503 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### njit with eager compilation and fastmath

In [17]:
@njit(float64[:,:](float64[:,:], float64[:]), fastmath=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.2264997959136963
Elapsed (after compilation) = 0.2265007495880127


In [18]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

231 ms ± 6.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### njit with parallel and fastmath

In [19]:
@njit(parallel=True, fastmath=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.8694961071014404
Elapsed (after compilation) = 0.010997772216796875


In [20]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

8.39 ms ± 683 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### njit with parallel and nogil

In [21]:
@njit(parallel=True, nogil=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.9234893321990967
Elapsed (after compilation) = 0.008499860763549805


In [22]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

8.59 ms ± 661 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### njit with nogil and fastmath

In [23]:
@njit(nogil=True, fastmath=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.561499834060669
Elapsed (after compilation) = 0.23550033569335938


In [24]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

231 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## All But One

### njit with eager compilation and nogil and parallel

In [25]:
@njit(float64[:,:](float64[:,:], float64[:]), nogil=True, parallel=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.012501239776611328
Elapsed (after compilation) = 0.010501623153686523


In [26]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

9.39 ms ± 676 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### njit with eager compilation and nogil and fastmath

In [27]:
@njit(float64[:,:](float64[:,:], float64[:]), nogil=True, fastmath=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.23499798774719238
Elapsed (after compilation) = 0.23649930953979492


In [28]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

241 ms ± 7.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### njit with eager compilation and parallel and fastmath

In [5]:
import time

@njit(float64[:,:](float64[:,:], float64[:]), parallel=True, fastmath=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.00749969482421875
Elapsed (after compilation) = 0.007500410079956055


In [6]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

7.84 ms ± 357 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### njit with parallel and nogil and fastmath

In [31]:
@njit(float64[:,:](float64[:,:], float64[:]), parallel=True, nogil=True, fastmath=True)
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.012001514434814453
Elapsed (after compilation) = 0.012499570846557617


In [32]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

8.09 ms ± 535 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## njit with eager compilation and parallel and nogil and fastmath

In [16]:
import time
from numba import int64

@njit(float64[:,:](float64[:,:], float64[:], float64, int32[:,:], int64), parallel=True, fastmath=True)
def compute_trunc_likes(x, mu, tau, bounds, which_param):
    
    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, np.sqrt(2))))) + (np.multiply(-0.5, erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, np.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.square(np.divide(x - mu, tau/np.sqrt(.5))))
    return log_likelihood

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param, tau, bounds, which_param)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param, tau, bounds, which_param)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.008002042770385742
Elapsed (after compilation) = 0.007998466491699219


In [17]:
%%timeit 
compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param, tau, bounds, which_param)

7.63 ms ± 374 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


So nogil probably does nothing but the rest of the options contribute some amount of performance. How much does this scale within the actual codebase?

## Integrating Into PCIT
I've already made an attempt, adding my vectorized `erfc` function with my new `compute_trunc_likes` and dependencies. Let's see if it works and compare performance with MATLAB.

First, let's time the MATLAB code.

In [12]:
tau.dtype

AttributeError: 'float' object has no attribute 'dtype'

## Still More to Do
I find that MATLAB gets 3795 iterations of compute_weights calls done in 5 minutes. Python is still only getting 938 iterations done. I might try going back to lower number precision. Could also try lesioning to find bottlenecks. That bug I detected as happening after all these iterations might also be an important clue.

```
Start time 1/4 3:8
********** START OF MESSAGES **********
0 trials are dropped since they are regarded as outliers
********** END OF MESSAGES **********
Betas: 0, 1
EM Iteration: 0
Optimization terminated successfully.
         Current function value: 1243.992529
         Iterations: 5
         Function evaluations: 7
         Gradient evaluations: 7
Betas: 0.10286141975053381, 0.6417252461065261
EM Iteration: 1
2021-01-04 03:12:00.616726
2021-01-04 03:17:11.770227
938
---------------------------------------------------------------------------
KeyboardInterrupt                         Traceback (most recent call last)
<ipython-input-6-1f3b069d409b> in compute_weights(curve_name, nParticles, normalized_w, prev_iter_curve_param, param, wgt_chunks, resolution)
     24                 nth_prev_iter_curve_param = prev_iter_curve_param[target_indices, npm]
---> 25                 trunc_likes = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param, tau, bounds, which_param)
     26                 prob_grp_lvl_curve = np.add(prob_grp_lvl_curve, trunc_likes)

KeyboardInterrupt: 

During handling of the above exception, another exception occurred:

AssertionError                            Traceback (most recent call last)
<ipython-input-9-2bc4507d9022> in <module>
      1 # run tests only when is main file!
      2 if __name__ == '__main__':
----> 3     test_importance_sampler()

<ipython-input-4-0f527f394b82> in test_importance_sampler()
     18 
     19     # generate output
---> 20     importance_sampler(python_data, python_analysis_settings)
     21     # eng.importance_sampler(matlab_data, matlab_analysis_settings, nargout=0)

<ipython-input-2-fd0875ea2295> in importance_sampler(***failed resolving arguments***)
    114         # Compute the p(theta) and q(theta) weights
    115         if em > 0:
--> 116             p_theta_minus_q_theta = compute_weights(
    117                 ana_opt['curve_type'], ana_opt['particles'], normalized_w[em - 1, :],
    118                 prev_iter_curve_param, param, ana_opt['wgt_chunks'], ana_opt['resolution'])

<ipython-input-6-1f3b069d409b> in compute_weights(curve_name, nParticles, normalized_w, prev_iter_curve_param, param, wgt_chunks, resolution)
     33         print(datetime.datetime.now())
     34         print(idx)
---> 35         assert(False)
     36 
     37     if np.any(np.isnan(q_theta)):

AssertionError: 
```